In [1]:
import numpy as np
import pandas as pd
from astropy.table import Table, vstack
from matplotlib import pyplot as plt, cm as cm, colors as colors, __version__ as mplver
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
from glob import glob
from astropy.timeseries import LombScargle
from astropy.table import Table
from astropy import __version__ as apver
from astropy import units as u, constants as c
from scipy import stats, __version__ as spver
import warnings
from TESStools import *
from tqdm.notebook import tqdm
from matplotlib import lines as mlines
from matplotlib.colors import Normalize
from scipy.stats import sigmaclip
import h5py as h5

In [2]:
print(np.__version__,pd.__version__,mplver,apver,spver)

1.20.3 1.3.0 3.3.4 4.2.1 1.6.2


In [3]:
#We're going to need our stars...
cool_sgs = pd.read_csv('sample.csv',index_col=0)
#merged with SLFV and prewhitening info...
slfv_params = pd.read_csv('slfv_params.csv',index_col=0)
inspected = pd.read_csv('prewhitening.csv',index_col=0)
merged = pd.merge(cool_sgs,slfv_params,left_index=True,right_index=True)
merged = pd.merge(merged,inspected,left_index=True,right_index=True)

# Now we're going to look for pulsators. First, grab stars where we can extract a frequency with >5 sigma

# Grab stars where we can extract a frequency with >5 sigma
pulse_detected = merged[(merged['n_peaks'] > 0) & (merged['highest_amp']/merged['highest_amp_error'] > 5)]
print(f"{len(pulse_detected)} stars with 5-sigma frequencies out of {len(merged)} total")

# We also used a GP to see if these frequencies were significant above the SLFV
gp_pulse = pd.read_csv('Find_FYPS_GP_results.csv', index_col=0)
pulse_detected_gp = pd.merge(pulse_detected, gp_pulse, left_index=True, right_index=True)
print(f"GP reveals {len(pulse_detected_gp[pulse_detected_gp['pulse_GP']==True])} true periodic stars")

# Load in info on which stars are consistent with RR Lyrae and Beta Cepheids...
likely_rrcep = pd.read_csv('likelyRRCep.csv',index_col=0)
likely_bcep = pd.read_csv('likelyBCep.csv',index_col=0)
contaminants = pd.merge(likely_rrcep,likely_bcep,left_index=True,right_index=True)

new = pd.merge(pulse_detected_gp,contaminants,left_index=True,right_index=True)
pulse_clean = new[(new['pulse_GP'])&(~new['LikelyRR'])&(~new['LikelyCep'])&(~new['LikelyBCep'])]
print(f"Filtering out contaminants leaves us with {len(pulse_clean)} stars")

155 stars with 5-sigma frequencies out of 201 total
GP reveals 154 true periodic stars
Filtering out contaminants leaves us with 111 stars


In [4]:
#A way to grab FYPS
def fyps_box(df):
    """
    Returns part of dataframe where teff and lum are in the FYPS box
    """
    
    #tmin = df['teff'] >= 3.6
    #tmax = df['teff'] <= 4.1
    #lmax = df['lum'] <= 7.76 - 0.6*df['teff']
    lmin = df['lum'] >= 5.0 #6.23 - 0.3*df['teff']
    
    return lmin # & lmax & tmin & tmax

In [5]:
candidate_fyps = pulse_clean[fyps_box(pulse_clean)]
all_in_box = merged[fyps_box(merged)]
len(candidate_fyps), len(all_in_box), len(candidate_fyps)/len(all_in_box)

(39, 63, 0.6190476190476191)

In [6]:
cf_lmc = candidate_fyps[candidate_fyps['Gal']=='LMC']
cf_smc = candidate_fyps[candidate_fyps['Gal']=='LMC']

In [67]:
mosaic = """
LLL
RRR
RRR
"""

for tic,cf in tqdm(candidate_fyps.iterrows(), total=len(candidate_fyps)):

    fig, ax_dict = plt.subplot_mosaic(mosaic, dpi=300)

    lc, lc_smooth = lc_extract(get_lc_from_id(tic), smooth='10T')
    time, flux, err = lc['Time'], lc['Flux'], lc['Err']
    t_s, f_s = lc_smooth['Time'], lc_smooth['Flux']

    ax_dict['L'].scatter(time, flux, s=1, c='k', rasterized=True)
    ax_dict['L'].plot(t_s, f_s, lw=0.5, c='C2', rasterized=True)
    ax_dict['L'].set(xlim=(time.min(),time.max()),
                     xlabel='Time',
                     ylabel='PDCSAP Flux')

    pseudo_NF = 0.5 / (np.mean(np.diff(time)))
    ls = LombScargle(time,flux,dy=err,)
    freq,power=ls.autopower(normalization='psd',maximum_frequency=pseudo_NF)
    power /= len(time)

    popt = cf[['alpha','tau','gamma','alphaw']].values
    fit = np.power(noise_func(freq,*popt),2.0)

    resid = power/fit

    ax_dict['R'].plot(freq,resid/resid.max(),rasterized=True) 
    
    with h5.File('prewhitening.hdf5', 'r') as f:
        good_fs = f[f'{tic}/good_fs'][()][:,0]
        
    for g in good_fs:
        plt.axvline(x=g, c='k', alpha=0.5, ls='--', zorder=-1)
    
    name = cf['CommonName'].lstrip('V*').replace(r'[',r'{[').replace(r']',r']}')
    if name[0:2] == 'SV':
        name = name.lstrip('SV*')
        
    
    ax_dict['R'].text(8,0.9,f'TIC {tic} \n {name}',rasterized=True)
    ax_dict['R'].set(xlim=(0,10),
                     xlabel='Frequency',
                     ylabel='Normalized Residual Power')

    fig.tight_layout()
    gal = cf['Gal'].lower()
    plt.savefig(f'../plots/app_plots/{gal}_tic{tic}_app.pdf', bbox_inches='tight')
    plt.close(fig)

  0%|          | 0/39 [00:00<?, ?it/s]

In [74]:
mosaic = """
LLL
RRR
RRR
"""

lmc_header = r"""\begin{deluxetable*}{lcccccc}
\tabletypesize{\scriptsize}
\tablecaption{Names, TIC numbers, coordinates, \tess magnitudes and positions in the HR diagram of the FYPS identified in the LMC, ordered by effective temperature from coolest to warmest. Typical uncertainties in $\log T_{\rm {eff}}$ and $\log L/L_\odot$ are 0.015 dex and 0.10 dex respectively.\label{tab:lmc_fyps}}
\tablehead{\colhead{Common Name} & \colhead{TIC Number} & \colhead{R.A.} & \colhead{Dec} & \colhead{$T$} & \colhead{$\log T_{\rm{eff}}$} & \colhead{$\log L/L_\odot$}\\
\colhead{} & \colhead{} & \colhead{[deg]} & \colhead{[deg]} & \colhead{[mag]} & \colhead{[K]} & \colhead{$L_\odot$}} 
\startdata
"""

smc_header = r"""\begin{deluxetable*}{lcccccc}
\tabletypesize{\scriptsize}
\tablecaption{Similar to Table \ref{tab:lmc_fyps} for FYPS identified in the SMC.\label{tab:smc_fyps}}
\tablehead{\colhead{Common Name} & \colhead{TIC Number} & \colhead{R.A.} & \colhead{Dec} & \colhead{$T$} & \colhead{$\log T_{\rm{eff}}$} & \colhead{$\log L/L_\odot$}\\
\colhead{} & \colhead{} & \colhead{[deg]} & \colhead{[deg]} & \colhead{[mag]} & \colhead{[K]} & \colhead{$L_\odot$}} 
\startdata
"""

lmc_plots_header = r"""\begin{figure*}[p!]
"""
smc_plots_header = r"""\begin{figure*}[p!]
"""

lmc_i = 0
lmc_num_box = 0
lmc_page_flag = False
smc_i = 0
smc_num_box = 0
smc_page_flag = False

for tic,cf in tqdm(candidate_fyps.iterrows(), total=len(candidate_fyps)):
    
    name = cf['CommonName'].lstrip('V*').replace(r'[',r'{[').replace(r']',r']}')
    if name[0:2] == 'SV':
        name = name.lstrip('SV*')

    gal = cf['Gal'].lower()
    
    # TeX
    if gal == 'lmc':
        if lmc_i % 2 == 0:
            lmc_plots_header += r'\gridline{\fig{'+f'app_plots/{gal}_tic{tic}_app.pdf'+r'}{0.5\textwidth}{}'
        elif lmc_i % 2 == 1:
            lmc_plots_header += r"""
            \fig{"""+f'app_plots/{gal}_tic{tic}_app.pdf'+r"""}{0.5\textwidth}{}}
            """+'\n'
        else:
            print('fucked up mod')
            
        ra = f"{cf['RA']:.8f}"
        dec = f"{cf['Dec']:.8f}"
        tmag = f"{cf['Tmag']:.3f}"
        teff = f"{cf['teff']:.3f}"
        lum = f"{cf['lum']:.3f}"

        entry = list(map(str,[name,tic,ra,dec,tmag,teff,lum]))

        sep = ' & '

        lmc_header += sep.join(entry)
        lmc_header += ' \\\\ \n'
        
        lmc_i += 1
        lmc_num_box += 1
        
        if (lmc_num_box == 6) & (not lmc_page_flag):
            lmc_plots_header.rstrip()
            lmc_plots_header += r"""\caption{Lightcurves and residual power spectra for LMC stars identified as FYPS. Top panels show the normalized \tess~PDCSAP lightcurve as black points, with a 10-minute rolling median in green. Bottom panels show the corresponding Lomb-Scargle periodogram, normalized by Equation \eqref{eq:rednoise} using best-fit parameters derived following \citet{dornwallenstein20b}. Frequencies identified via prewhitening are shown as vertical dashed lines.}\figurenum{1}\label{fig:lmc_fyps}
\end{figure*}
            
\begin{figure*}[p!]
            """
            lmc_num_box = 0
            lmc_page_flag = True
        elif (lmc_num_box == 6) & lmc_page_flag:
            lmc_plots_header.rstrip()
            lmc_plots_header += r"""\caption{{\it cont.}}\figurenum{1}
\end{figure*}
            
\begin{figure*}[p!]
            """
            lmc_num_box = 0
            lmc_page_flag = True
        
    elif gal == 'smc':
        if smc_i % 2 == 0:
            smc_plots_header += r'\gridline{\fig{'+f'app_plots/{gal}_tic{tic}_app.pdf'+r'}{0.5\textwidth}{}'
        elif smc_i % 2 == 1:
            smc_plots_header += r"""
            \fig{"""+f'app_plots/{gal}_tic{tic}_app.pdf'+r"""}{0.5\textwidth}{}}
            """+'\n'
        else:
            print('fucked up mod')
            
        ra = f"{cf['RA']:.8f}"
        dec = f"{cf['Dec']:.8f}"
        tmag = f"{cf['Tmag']:.3f}"
        teff = f"{cf['teff']:.3f}"
        lum = f"{cf['lum']:.3f}"

        entry = list(map(str,[name,tic,ra,dec,tmag,teff,lum]))

        sep = ' & '

        smc_header += sep.join(entry)
        smc_header += ' \\\\ \n'
        
        smc_i += 1
        smc_num_box += 1
        
        if (smc_num_box == 6) & (not smc_page_flag):
            smc_plots_header.rstrip()
            smc_plots_header += r"""\caption{Similar to Figure \ref{fig:lmc_fyps} for FYPS identified in the SMC.}\figurenum{2}\label{fig:smc_fyps}
\end{figure*}
            
\begin{figure*}[p!]
            """
            smc_num_box = 0
            smc_page_flag = True
        elif (smc_num_box == 6) & smc_page_flag:
            smc_plots_header.rstrip()
            smc_plots_header += r"""\caption{{\it cont.}}\figurenum{2}
\end{figure*}
            
\begin{figure*}[p!]
            """
            smc_num_box = 0
            smc_page_flag = True
        
    else:
        print('fucked up gal')
        
lmc_header.rstrip()
lmc_header.rstrip('\\')
lmc_header.rstrip()
lmc_header += r"""\enddata
\end{deluxetable*}

"""

smc_header.rstrip()
smc_header.rstrip('\\')
smc_header.rstrip()
smc_header += r"""\enddata
\end{deluxetable*}

"""

if lmc_i % 2 == 1:
    lmc_plots_header += '}\n'
    
if smc_i % 2 == 1:
    smc_plots_header += '}\n'
    
lmc_plots_header.rstrip()
lmc_plots_header += r"""\caption{{\it cont.}}\figurenum{1}
            \end{figure*}
            """

smc_plots_header.rstrip()
smc_plots_header += r"""\caption{{\it cont.}}\figurenum{2}
            \end{figure*}
            """

with open("apptable_LMC.tex", "w") as text_file:
    text_file.write(lmc_header)
    
with open("appplot_LMC.tex", "w") as text_file:
    text_file.write(lmc_plots_header)
        
with open("apptable_SMC.tex", "w") as text_file:
    text_file.write(smc_header)
    
with open("appplot_SMC.tex", "w") as text_file:
    text_file.write(smc_plots_header)

  0%|          | 0/39 [00:00<?, ?it/s]